In [1]:
import numpy as np
import re
import json
import pandas as pd
import warnings
import csv
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
%load_ext autoreload
%autoreload 2
import label_improve as li

In [3]:
# Loading the data 
dataset_name = "chemprot"
idx_to_label = json.load(open(f"label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
test_df = li.data_to_df(json.load(open(f"test.json", "r")))

In [4]:
test_df = test_df.sample(250, random_state=205)
eval_df = test_df.drop(columns=[ 'label', 'weak_labels'])

In [5]:
eval_df.to_csv('./chemprot_eval.csv')

In [6]:
from openai import OpenAI
import os
import sys

API_KEY = ''


model_id = 'gpt-4o-2024-08-06' # replace with your model ID

os.environ["OPENAI_API_KEY"] = API_KEY

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [19]:
import openai
import pandas as pd

# Load your CSV file
df = pd.read_csv('chemprot_eval.csv')


# Define the categories
categories = {
    "0": "Part of",
    "1": "Regulator",
    "2": "Upregulator",
    "3": "Downregulator",
    "4": "Agonist",
    "5": "Antagonist",
    "6": "Modulator",
    "7": "Cofactor",
    "8": "Substrate/Product",
    "9": "NOT"
}

# Function to classify text
def classify_text(text):
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": "You are an expert in biochemistry and molecular biology."},
            {"role": "user", "content": f"Classify the following text into one of the categories: {categories}. Text: {text}, also provide the label number at the end of the sentences, with the exact sentence 'The label is : $number'"}
        ]
    )
    classification = response.choices[0].message.content
    return classification.strip()



# Apply the function to each line of text
df['Classification'] = df['text'].apply(classify_text)

# Save the results to a new CSV file
df.to_csv('classified_file.csv', index=False)


In [27]:
import re


# Extract the label number at the end of the 'Classification' column
df['Classification'] = df['Classification'].str.extract(r'The label is\s*:\s*(\d+)')
 
df.to_csv('classified_file2.csv', index=False)


In [28]:
predictions_df = pd.read_csv('classified_file2.csv')

true_labels_df =  test_df

merged_df = pd.merge(predictions_df, true_labels_df, on='text')

merged_df['correct'] = merged_df['Classification'] == merged_df['label']

accuracy = merged_df['correct'].mean()
print(f"Accuracy: {accuracy:.2%}")

print(merged_df.head())

merged_df.to_csv('comparison_with_true_labels.csv', index=False)


Accuracy: 42.64%
   Unnamed: 0                                               text  \
0       15300  Fragment-based drug design and identification ...   
1        7570  Administration of cevimeline hydrochloride, an...   
2       11967  PIP5K1B encodes phosphatidylinositol 4-phospha...   
3       13674  Both porcine TLR7 and TLR8 proteins were expre...   
4        1411  RATIONALE: There is substantial evidence that ...   

   Classification  label                                        weak_labels  \
0               5      3  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
1               4      9  [-1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, 3,...   
2               8      8  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
3               0      0  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   
4               4      1  [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...   

   correct  
0    False  
1    False  
2     True  
3     True  
4    False  
